# Library

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [5]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

# 페이지 접속

In [6]:
# 페이지 주소
url = 'https://news.naver.com/section/100' # 네이버 뉴스 > 정치

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [ ]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

<Response [200]>


'정치 : 네이버 뉴스'

## selenium 작동

In [8]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

## 헤드라인 뉴스 섹션 로드

In [9]:
# 헤드라인 뉴스
headline_news_more_view_button_xpath = '//*[@id="newsct"]/div[1]/div[2]/a' # 헤드라인 더 보기 버튼

# 헤드라인 더 보기 버튼 로딩 대기
headline_news_more_view_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, headline_news_more_view_button_xpath))
)

# 헤드라인 더보기 버튼 클릭
headline_news_more_view_button.click()
driver.implicitly_wait(10)

# 헤드라인 뉴스 수집

In [10]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'정치 : 네이버 뉴스'

In [11]:
# 헤드라인 뉴스 섹션
headline_news_section = soup.select_one('ul.sa_list')
len(headline_news_section)

21

In [12]:
# 뉴스 타이들 리스트
headline_news_list = headline_news_section.select('li.sa_item._SECTION_HEADLINE')
len(headline_news_list)

10

## 기사 내용 섹션

In [13]:
# 기사 내용 섹션
order_number = 0
news_content_section = headline_news_list[order_number].select_one('div.sa_text')
news_content_section

<div class="sa_text">
<a class="sa_text_title _NLOG_IMPRESSION" data-clk="pol.clart" data-imp-gdid="88000114_000000000000000011861987" data-imp-index="1" data-imp-url="https://n.news.naver.com/mnews/article/056/0011861987" href="https://n.news.naver.com/mnews/article/056/0011861987">
<strong class="sa_text_strong">민주 “경찰차벽 철수, 농민 투쟁 결과…‘입틀막’은 불행한 역사”</strong>
</a>
<div class="sa_text_lede">대통령 한남동 관저를 향하는 전국농민회총연맹(전농) 트랙터를 막아서던 경찰 차벽이 철수한 것에 대해 더불어민주당이 “윤석열식 ‘입틀막 공권력 행사’가 더는 반복돼서는 안 된다”고 밝혔습니다. 민주당 김성회 대변인은 </div>
<div class="sa_text_info">
<div class="sa_text_info_left">
<div class="sa_text_press">KBS</div>
<a class="sa_text_cmt _COMMENT_COUNT_LIST" data-object-id="news056,0011861987" data-processed="true" data-ticket="news" data-zero-allow="false" href="https://n.news.naver.com/mnews/article/comment/056/0011861987" style="">10<span class="sa_text_symbol"><span class="blind">이상</span>+</span></a>
</div>
<div class="sa_text_info_right">
<a class="sa_text_cluster" data-clk="clcou" href=

### 기사 링크

In [14]:
# 기사 링크
link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
link

'https://n.news.naver.com/mnews/article/056/0011861987'

### 타이틀

In [15]:
# 타이틀
title = news_content_section.select_one('strong.sa_text_strong').text
title

'민주 “경찰차벽 철수, 농민 투쟁 결과…‘입틀막’은 불행한 역사”'

### 기사 내용

In [16]:
# 기사 내용
content = news_content_section.select_one('div.sa_text_lede').text
content

'대통령 한남동 관저를 향하는 전국농민회총연맹(전농) 트랙터를 막아서던 경찰 차벽이 철수한 것에 대해 더불어민주당이 “윤석열식 ‘입틀막 공권력 행사’가 더는 반복돼서는 안 된다”고 밝혔습니다. 민주당 김성회 대변인은 '

### 언론사

In [17]:
# 언론사
press = news_content_section.select_one('div.sa_text_press').text
press

'KBS'

### 관련 뉴스 개수

In [18]:
# 관련 뉴스 개수
related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
related_news_count

10

# 수집한 데이터 전처리

In [19]:
link_list = []
title_list = []
content_list = []
press_list = []
related_news_count_list = []

for i in range(len(headline_news_list)):
    # 기사 내용 섹션
    news_content_section = headline_news_list[i].select_one('div.sa_text')

    # 기사 링크
    try:
        link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
    except:
        link = np.nan
        print(i)
        # print('기사 링크 없음')
    
    # 타이틀
    try:
        title = news_content_section.select_one('strong.sa_text_strong').text
    except:
        title = np.nan
        print(i)
        # print('타이틀 없음')
    
    # 기사 내용
    try:
        content = news_content_section.select_one('div.sa_text_lede').text
    except:
        content = np.nan
        print(i)
        # print('기사 내용 없음')
    
    ## 언론사
    try:
        press = news_content_section.select_one('div.sa_text_press').text
    except:
        press = np.nan
        print(i)
        # print('언론사 없음')
    
    ## 관련 뉴스 개수
    try:
        related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
    except:
        related_news_count = np.nan
        print(i)
        # print('관련 뉴스 개수 없음')
    
    # 정보 리스트에 담기
    link_list.append(link)
    title_list.append(title)
    content_list.append(content)
    press_list.append(press)
    related_news_count_list.append(related_news_count)

In [20]:
len(link_list), len(title_list), len(content_list), len(press_list), len(related_news_count_list)

(10, 10, 10, 10, 10)

In [21]:
data_dict = {
    '번호': range(1, len(link_list)+1),
    '기사 링크': link_list,
    '기사 제목': title_list,
    '기사 내용': content_list,
    '언론사': press_list,
    '관련 뉴스 개수': related_news_count_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,기사 링크,기사 제목,기사 내용,언론사,관련 뉴스 개수
0,1,https://n.news.naver.com/mnews/article/056/001...,"민주 “경찰차벽 철수, 농민 투쟁 결과…‘입틀막’은 불행한 역사”",대통령 한남동 관저를 향하는 전국농민회총연맹(전농) 트랙터를 막아서던 경찰 차벽이 ...,KBS,10
1,2,https://n.news.naver.com/mnews/article/052/000...,"박찬대 ""한 대행, 24일까지 특검법 공포 않으면 책임 물을 것""",더불어민주당 박찬대 원내대표가 한덕수 대통령 권한대행을 향해 오는 24일까지 '내란...,YTN,134
2,3,https://n.news.naver.com/mnews/article/658/000...,與 비대위원장 중진 현역 가닥…‘도로 친윤당’ 딜레마,- 유승민·윤희숙 파격인선 주장도 국민의힘이 이번 주 비상대책위원장 인선을 놓고 ‘...,국제신문,25
3,4,https://n.news.naver.com/mnews/article/214/000...,"국민의힘 ""'이재명 안 됩니다' 불허 선관위‥편파적 유권 해석""","중앙선거관리위원회가 국민의힘 의원들을 '내란공범'으로 지칭한 현수막은 허용하고, '...",MBC,106
4,5,https://n.news.naver.com/mnews/article/082/000...,야당 압박에 또 고민 깊어진 한덕수,한덕수 대통령 권한대행의 고민이 또다시 깊어졌다. 야권이 ‘내란·김건희 여사 특검법...,부산일보,36


# 데이터 출력

In [22]:
current_date = time.strftime('%Y%m%d')
current_date

'20241223'

In [23]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241223_AM_120544'

In [24]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [29]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_news', 'politics', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\naver_news\\politics\\20241223'

In [30]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [31]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_news_politics_{current_datetime}.xlsx'), index=False)

In [28]:
driver.quit()

# END